### BioGRID interaction retrieval

In [1]:
import requests, sys, json
import pandas as pd
import pickle

In [2]:
### BioGRID access key
BIOGRID_ACCESS_KEY = 'a207ffa23db79e55a7d7a5a69b79f0c9'

### BioGRID server
biogrid_server = f'https://webservice.thebiogrid.org/interactions/?accesskey={BIOGRID_ACCESS_KEY}'

In [13]:
with open('../Results/GENIE3/SARS-CoV-2/interactions_dataframe_with_entrez.pickle', 'rb') as f:
    interactions = pickle.load(f)

interactions

,gene1,gene2,score,gene1_name,gene2_name,gene1_entrez,gene2_entrez
0,ENSG00000200336,ENSG00000201695,0.102868,5S_rRNA,5S_rRNA,106480763,106865375
1,ENSG00000201695,ENSG00000200336,0.085720,5S_rRNA,5S_rRNA,106865375,106480763
2,ENSG00000273590,ENSG00000205670,0.081955,LOC102723553,LOC102723553,102723553,54065
3,ENSG00000205670,ENSG00000273590,0.077385,LOC102723553,LOC102723553,54065,102723553
4,ENSG00000160201,ENSG00000275895,0.072684,LOC102724594,LOC102724594,7307,102724594
...,...,...,...,...,...,...,...
249495,ENSG00000137673,ENSG00000201965,0.000000,MMP7,Y_RNA,4316,None
249496,ENSG00000206918,ENSG00000199546,0.000000,U6,Y_RNA,106480085,None
249497,ENSG00000252714,ENSG00000240665,0.000000,5S_rRNA,LSP1P2,100873645,100873967
249498,ENSG00000228078,ENSG00000202354,0.000000,HLA-U,RNY3,None,6085


In [14]:
top_ints = interactions[['gene1_entrez', 'gene2_entrez']].iloc[:100].values
top_ints

array([['106480763', '106865375'],
       ['106865375', '106480763'],
       ['102723553', '54065'],
       ['54065', '102723553'],
       ['7307', '102724594'],
       ['283955', None],
       ['100008587', '109910384'],
       ['109910381', '124907150'],
       ['100271063', None],
       ['3120', '388344'],
       ['100008587', '109864274'],
       [None, None],
       ['109864274', '109910381'],
       ['109864281', '124907150'],
       ['55258', '474384'],
       ['124907150', '109864281'],
       ['124907150', '106632260'],
       ['124907150', '124907150'],
       ['109864281', '109910384'],
       [None, None],
       ['109910384', '109910381'],
       ['106632260', '109910384'],
       ['7503', '8287'],
       ['109864274', '100008587'],
       [None, '283955'],
       ['105376805', None],
       [None, None],
       ['8363', '8362'],
       ['109910381', '100008587'],
       [None, '7544'],
       ['57834', '729654'],
       ['109864274', '124907150'],
       ['109910381', '1

In [16]:
gene1, gene2 = top_ints[1]
print(gene1, gene2)
# for gene1, gene2 in top_ints:
# the rules outlined in the Wiki: https://wiki.thebiogrid.org/doku.php/biogridrest
params = {
    "accesskey": BIOGRID_ACCESS_KEY,
    "format": "json",  # Return results in TAB2 format
    "geneList": "|".join(gene1),  # Must be | separated
    "searchNames": "true",  # Search against official names
    "includeInteractors": "false",  # Set to true to get any interaction involving EITHER gene, set to false to get interactions between genes
    "includeHeader": "true",
}

# Additional options to try, you can uncomment them as necessary
# params["start"] = 5 # Specify where to start fetching results from if > 10,000 results being returned
# params["max"] = 10 # Specify the number of results to return, max is 10,000
# params["interSpeciesExcluded"] = "false" # true or false, If ‘true’, interactions with interactors from different species will be excluded (ex. no Human -> Mouse interactions)
# params["selfInteractionsExcluded"] = "false" # true or false, If ‘true’, interactions with one interactor will be excluded. (ex. no STE11 -> STE11 interactions)
# params["searchIds"] = "true" # true or false, If ‘true’, ENTREZ_GENE, ORDERED LOCUS and SYSTEMATIC_NAME (orf) will be examined for a match with the geneList
# params["searchSynonyms"] = "false" # true or false, If ‘true’, SYNONYMS will be examined for a match with the geneList
# params["searchBiogridIds"] = "false" # true or false, If ‘true’, BIOGRID INTERNAL IDS will be examined for a match with the geneList
# params["excludeGenes"] = "false" # true or false, If 'true' the geneList becomes a list of genes to EXCLUDE rather than to INCLUDE
# params["includeInteractorInteractions"] = "true" # true or false, If ‘true’ interactions between the geneList’s first order interactors will be included. Ignored if includeInteractors is ‘false’ or if excludeGenes is set to ‘true’.
# params["htpThreshold"] = 50 # Any publication with more than this many interactions will be excluded
# params["throughputTag"] = "any" # any, low, high. If set to low, only `low throughput` interactions will be returned, if set to high, only `high throughput` interactions will be returned
# params["additionalIdentifierTypes"] = "SGD|FLYBASE|REFSEQ" # You can specify a | separated list of additional identifier types to search against (see get_identifier_types.py)

r = requests.get(biogrid_server, params=params)
interactions = r.json()

# Create a hash of results by interaction identifier
data = {}
for interaction_id, interaction in interactions.items():
    data[interaction_id] = interaction
    # Add the interaction ID to the interaction record, so we can reference it easier
    data[interaction_id]["INTERACTION_ID"] = interaction_id

# Load the data into a pandas dataframe
dataset = pd.DataFrame.from_dict(data, orient="index")

columns = [
    "INTERACTION_ID",
    "SYSTEMATIC_NAME_A",
    "SYSTEMATIC_NAME_B",
    "OFFICIAL_SYMBOL_A",
    "OFFICIAL_SYMBOL_B",
    "EXPERIMENTAL_SYSTEM",
    "PUBMED_ID",
    "PUBMED_AUTHOR",
    "THROUGHPUT",
    "QUALIFICATIONS",
]
# dataset = dataset[columns]
dataset


106865375 106480763


AttributeError: 'list' object has no attribute 'items'